In [10]:
import json
src_json_path = "Tianbao_annotation_fix_3.json"
dst_json_path = "annotations_v5.json"

def convert_format(src_data):
    result = []

    for i, annotation in enumerate(src_data["annotations"]):

        image_size = src_data["image"]["size"]
        image_size = image_size if image_size[0]>image_size[1] else [image_size[1], image_size[0]] 
        converted_item = {
            "id": f"{src_data['id']}-{i}",
            "image_path": src_data["image"]["path"],
            "image_size": image_size,
            "instruction": annotation["attributes"]["Instruction"],
            "box_type": annotation["type"],
            "box_coordinates": annotation["bbox"] if annotation["type"] == "bbox" else annotation["points"]
        }
        result.append(converted_item)

    return result

# Sample src data - replace this with your actual data
with open(src_json_path, "r") as file:
    file_data = json.load(file)
    src_data_list = file_data["items"]

with open(dst_json_path, "r")as file:
    dst_data_list = json.load(file)
    print(f"original length: {len(dst_data_list)}")
    # iterate on id
    for src_data in src_data_list:

        src_id = src_data["id"]
        converted_data = convert_format(src_data)
        # delete
        indices_to_replace = [i for i, dst_data in enumerate(dst_data_list) if src_id in dst_data["id"]]

        # 如果找到了满足条件的元素，进行替换
        if indices_to_replace:
            print(1)
            # 删除满足条件的元素
            for index in reversed(indices_to_replace):
                del dst_data_list[index]
            # 插入 converted_data 中的元素
            dst_data_list[indices_to_replace[0]:indices_to_replace[0]] = converted_data


with open(dst_json_path, "w")as file:
    print(f"now length: {len(dst_data_list)}")
    json.dump(dst_data_list, file, indent=2)

original length: 566
1
1
1
now length: 566


In [4]:
import json
with open("annotations_v5.json", "r") as file:
    data_list = json.load(file)
    for data in data_list:
        if data["image_size"][0]<data["image_size"][1]:
            data["image_size"] = [data["image_size"][1], data["image_size"][0]]
            if data["image_size"][0] != 1920 and data["image_size"][0] != 1280:
                print(data["image_size"])
with open("annotations_v5.json", "w") as file:
    json.dump(data_list, file, indent=2)

In [13]:
import json
# 找到所有image_path相同且instruction相同的data
anno_json_path = "annotations_v5.json"
with open(anno_json_path, "r") as file:
    anno_data_list = json.load(file)
for i, anno_data_i in enumerate(anno_data_list):
    for j, anno_data_j in enumerate(anno_data_list[i+1:]):
        if anno_data_i["image_path"] == anno_data_j["image_path"] and anno_data_i["instruction"] == anno_data_j["instruction"]:
            print(str(anno_data_i), str(anno_data_j))

{'id': 'e9jRS50WhC-0', 'image_path': 'e9jRS50WhC.png', 'image_size': [1280, 720], 'instruction': 'Close the opened menu', 'box_type': 'bbox', 'box_coordinates': [850.99, 595.67, 115.59000000000003, 30.700000000000045]} {'id': 'e9jRS50WhC-1', 'image_path': 'e9jRS50WhC.png', 'image_size': [1280, 720], 'instruction': 'Close the opened menu', 'box_type': 'polygon', 'box_coordinates': [957.94, 118.49, 951.38, 121.11, 948.36, 126.5, 948.88, 133.32, 953.08, 137.78, 960.7, 139.2, 967.1, 134.1, 968, 125.7, 964.11, 120.46]}


In [15]:
import os
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from PIL import Image, ImageDraw, ImageFont
# 找到所有image_path相同且instruction相同的data
anno_json_path = "annotations_v5.json"
img_dir = "images"
vis_dir = "annotations_v5_vis"
os.makedirs(vis_dir, exist_ok=True)
with open(anno_json_path, "r") as file:
    anno_data_list = json.load(file)

def visualize(anno_data):
    img_path = os.path.join(img_dir, anno_data["image_path"])
    vis_path = os.path.join(vis_dir, f"{anno_data['id']}.png")
    # annotate:

    image_size = anno_data["image_size"]
    image = Image.open(img_path)
    draw = ImageDraw.Draw(image)

    if anno_data["box_type"] == "bbox":
        box_coordinates = (anno_data["box_coordinates"][0], anno_data["box_coordinates"][1], anno_data["box_coordinates"][0]+anno_data["box_coordinates"][2], anno_data["box_coordinates"][1]+anno_data["box_coordinates"][3])
        draw.rectangle(box_coordinates, outline="green", width=2)
    elif anno_data["box_type"] == "polygon":
        polygon_coordinates = anno_data["box_coordinates"]
        draw.polygon(polygon_coordinates, outline="green")
    elif anno_data["box_type"] == "refusal":
        pass
    else:
        raise ValueError(f"box_type must be 'bbox' or 'polygon', not{anno_data['box_type']}")
    # 定义指令文本
    instruction = anno_data["instruction"] + "  " + anno_data[""]
    if anno_data["box_type"] == "refusal":
        instruction += "[refusal]"
    font = ImageFont.load_default(size=30)

    # 计算文本的尺寸
    left, top, right, bottom = font.getbbox(instruction)
    text_width = right - left
    text_height = bottom - top

    # 定义文本位置
    text_position = (40, image_size[1] - 40)

    # 绘制白色背景矩形
    background_position = (
        text_position[0],  # x
        text_position[1],  # y
        text_position[0] + text_width,  # x + width
        text_position[1] + text_height  # y + height
    )
    draw.rectangle(background_position, fill="white")
    draw.text(text_position, instruction, fill="black", font=font)

    gui_types = str(anno_data["GUI_types"])
    font = ImageFont.load_default(size=30)

    # 计算文本的尺寸
    left, top, right, bottom = font.getbbox(gui_types)
    text_width = right - left
    text_height = bottom - top

    # 定义文本位置
    text_position = (40, 40)

    # 绘制白色背景矩形
    background_position = (
        text_position[0],  # x
        text_position[1],  # y
        text_position[0] + text_width,  # x + width
        text_position[1] + text_height  # y + height
    )
    draw.rectangle(background_position, fill="white")
    draw.text(text_position, gui_types, fill="black", font=font)
    # 保存图像
    image.save(vis_path)

with ThreadPoolExecutor(max_workers=20) as executor:
    futures = [executor.submit(visualize, anno_data) for anno_data in anno_data_list]
    for future in as_completed(futures):
        future.result()


In [12]:
# 对比有GUI types的和没有GUI types的
withtypes_path = "annotations_v5.json"
notypes_path = "annotations_v5_notypes.json"
with open(withtypes_path, "r") as file:
    withtypes_data_list = json.load(file)
with open(notypes_path, "r") as file:
    notypes_data_list = json.load(file)
for i, withtypes_data_i in enumerate(withtypes_data_list):
    data_id = withtypes_data_i["id"]
    same_exists = False
    for j, notypes_data_j in enumerate(notypes_data_list):
        if notypes_data_j["id"] == data_id:
            same_exists = True
            if withtypes_data_i["instruction"] != notypes_data_j["instruction"] or withtypes_data_i["box_coordinates"] != notypes_data_j["box_coordinates"]:
                print("same id but different data")
                print(str(withtypes_data_i), str(notypes_data_j))
    if same_exists == False:
        print("no same id")
        print(str(withtypes_data_i))